## Deep Learning Adaptation with Word Embeddings for Sentiment Analysis on Online Course Reviews

In [1]:
from keras.layers import RepeatVector, dot, LSTM, Permute, merge, TimeDistributed, Bidirectional, Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, GaussianNoise
from keras.layers import Activation, MaxoutDense, Embedding, SimpleRNN, MaxoutDense
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras.constraints import maxnorm
from keras.models import load_model
from keras.models import Sequential
from sklearn.manifold import TSNE
from keras.optimizers import Adam
from keras.regularizers import l2
from sklearn import preprocessing
from keras import backend as K
from keras import initializers
from keras import regularizers
from keras import constraints
import pandas as pd
import numpy as np
import argparse
import time
import sys
import csv
import os

Using TensorFlow backend.


### 1. Load and Plot Word Embeddings

In [2]:
embedding_path = 'embeddings/specific/word2vec/word2vec_300_50.txt'

In [3]:
embeddings_index = {}
with open(embedding_path) as embedding_file:
    for line in embedding_file:
        values = line.split(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
print('Sample embedding for "fantastic":', embeddings_index['fantastic'][:6], '...')

In [ ]:
embeddings_values, embeddings_words = np.array(list(embeddings_index.values())), np.array(list(embeddings_index.keys()))
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(embeddings_values)

In [ ]:
plt.rcParams.update({'font.size': 16.5})
plt.figure(figsize=(10, 4))

term_1 = 'fantastic'
term_2 = 'great'
term_3 = 'fantastic'

index_term_1, = np.where(embeddings_words == term_1)
index_term_2, = np.where(embeddings_words == term_2)
index_term_3, = np.where(embeddings_words == term_3)

plt.title(r'Sample Word Embeddings')
plt.scatter(tsne_results[index_term_1,0], tsne_results[index_term_1,1], s=np.pi*10, linewidth=0.5, edgecolor='black', alpha=0.25, label=term_1)
plt.scatter(tsne_results[index_term_2,0], tsne_results[index_term_2,1], s=np.pi*10, linewidth=0.5, edgecolor='black', alpha=0.25, label=term_2)
plt.scatter(tsne_results[index_term_3,0], tsne_results[index_term_3,1], s=np.pi*10, linewidth=0.5, edgecolor='black', alpha=0.25, label=term_3)
plt.xticks([])
plt.yticks([])
plt.legend(framealpha=0.5)

plt.tight_layout()
plt.show()

### 2. Load Models and Predict Sentiment Scores

In [ ]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class Attention(Layer):

    def __init__(self, W_regularizer=None, b_regularizer=None, W_constraint=None, b_constraint=None, bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],), initializer=self.init, name='{}_W'.format(self.name), regularizer=self.W_regularizer, constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[1],), initializer='zero', name='{}_b'.format(self.name), regularizer=self.b_regularizer, constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        eij = dot_product(x, self.W)

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def get_output_shape_for(self, input_shape):
        return input_shape[0], input_shape[-1]

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
model_path = 'models/class2/word2vec_300_30_fold0_model.h5'

In [ ]:
model = load_model(model_path, {'Attention': Attention, 'dot_product': dot_product})

In [ ]:
test_review = 'Bad course'

In [ ]:
max_len = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
all_words = [word for word, i in word_index.items()]
max_words = len(word_index) + 1
print('Found %s unique words.' % len(word_index))

data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

print('* PRE-PROCESSING DATA')
skf = StratifiedKFold(n_splits=args.n_fold)

print('Scaling score values')
if args.n_classes == 10:
    discrete_labels = np.array([getClass10(x) for x in labels])
elif args.n_classes == 5:
    discrete_labels = np.array([getClass5(x) for x in labels])
else:
    discrete_labels = np.array([getClass2(x) for x in labels])

print('Shuffling data')
s = np.arange(data.shape[0])
np.random.shuffle(s)
data = data[s]

In [ ]:
y_pred = model.predict([test_review])

In [ ]:
print('The sentiment score for "', test_review, '" is', y_pred[0])